Google Colab link : https://colab.research.google.com/drive/120IqNdb_44K1kSgqTTsZosf7VOaLO0ki?usp=sharing


# Preliminaries

In [1]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

Dataset link : https://www.kaggle.com/datasets/tunguz/big-five-personality-test

In [3]:
dataset = pd.read_csv("/content/drive/MyDrive/SEM4/Research Method/RM Kel 19 Experiment/data-final.csv", sep='\t')

print(dataset.shape)
dataset.head()

(1015341, 110)


,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,...,dateload,screenw,screenh,introelapse,testelapse,endelapse,IPC,country,lat_appx_lots_of_err,long_appx_lots_of_err
0,4.0,1.0,5.0,2.0,5.0,1.0,5.0,2.0,4.0,1.0,...,2016-03-03 02:01:01,768.0,1024.0,9.0,234.0,6,1,GB,51.5448,0.1991
1,3.0,5.0,3.0,4.0,3.0,3.0,2.0,5.0,1.0,5.0,...,2016-03-03 02:01:20,1360.0,768.0,12.0,179.0,11,1,MY,3.1698,101.706
2,2.0,3.0,4.0,4.0,3.0,2.0,1.0,3.0,2.0,5.0,...,2016-03-03 02:01:56,1366.0,768.0,3.0,186.0,7,1,GB,54.9119,-1.3833
3,2.0,2.0,2.0,3.0,4.0,2.0,2.0,4.0,1.0,4.0,...,2016-03-03 02:02:02,1920.0,1200.0,186.0,219.0,7,1,GB,51.75,-1.25
4,3.0,3.0,3.0,3.0,5.0,3.0,3.0,5.0,3.0,4.0,...,2016-03-03 02:02:57,1366.0,768.0,8.0,315.0,17,2,KE,1.0,38.0


# Data Preprocessing

In [20]:
data = dataset.drop(list(dataset)[50:], axis=1)

print(data.shape)
data.head()

(1015341, 50)


,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,...,OPN1,OPN2,OPN3,OPN4,OPN5,OPN6,OPN7,OPN8,OPN9,OPN10
0,4.0,1.0,5.0,2.0,5.0,1.0,5.0,2.0,4.0,1.0,...,5.0,1.0,4.0,1.0,4.0,1.0,5.0,3.0,4.0,5.0
1,3.0,5.0,3.0,4.0,3.0,3.0,2.0,5.0,1.0,5.0,...,1.0,2.0,4.0,2.0,3.0,1.0,4.0,2.0,5.0,3.0
2,2.0,3.0,4.0,4.0,3.0,2.0,1.0,3.0,2.0,5.0,...,5.0,1.0,2.0,1.0,4.0,2.0,5.0,3.0,4.0,4.0
3,2.0,2.0,2.0,3.0,4.0,2.0,2.0,4.0,1.0,4.0,...,4.0,2.0,5.0,2.0,3.0,1.0,4.0,4.0,3.0,3.0
4,3.0,3.0,3.0,3.0,5.0,3.0,3.0,5.0,3.0,4.0,...,5.0,1.0,5.0,1.0,5.0,1.0,5.0,3.0,5.0,5.0


In [21]:
for i in data.columns:
  data = data[(data[i].notna()) & (data[i] != 0)]

print(data.shape)

(874434, 50)


In [22]:
x = data.drop(columns=['EST9'])
y = data['EST9']

## Random oversampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
data['EST9'].value_counts()

4.0    247851
2.0    199050
3.0    182001
5.0    133152
1.0    112380
Name: EST9, dtype: int64

In [ ]:
oversampler = RandomOverSampler(random_state=42)

In [ ]:
x_resampled, y_resampled = oversampler.fit_resample(x, y)

In [ ]:
y_resampled.value_counts()

3.0    247851
1.0    247851
4.0    247851
2.0    247851
5.0    247851
Name: EST9, dtype: int64

## XGBoost feature selection

In [24]:
x = data[['EST6', 'EST8', 'AGR3', 'EST7', 'EST1', 'EST5', 'CSN9', 'EST10', 'AGR6', 'EST3']]

## Shaping dataset

In [25]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)

In [26]:
timesteps = 1
input_dim = 10

x_1 = np.resize(x, (x.shape[0], timesteps, input_dim))
x_train_1 = np.resize(x_train, (x_train.shape[0], timesteps, input_dim))
x_test_1 = np.resize(x_test, (x_test.shape[0], timesteps, input_dim))

In [27]:
y_train_1 = y_train - 1
y_test_1 = y_test - 1

In [73]:
y_train_1 = to_categorical(y_train_1)
y_test_1 = to_categorical(y_test_1)

print(y_test_1)

[[0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 ...
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]]


# LSTM training

In [29]:
x_train_1.shape

(786990, 1, 10)

In [32]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [33]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [34]:
model.fit(x_train_1, y_train_1, epochs=10, batch_size=256, validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', patience=7, min_delta=0.0001)])

Epoch 1/10
2460/2460 [==============================] - 23s 7ms/step - loss: 1.2541 - accuracy: 0.4604 - val_loss: 1.1909 - val_accuracy: 0.4944
Epoch 2/10
2460/2460 [==============================] - 18s 7ms/step - loss: 1.2139 - accuracy: 0.4841 - val_loss: 1.1767 - val_accuracy: 0.5011
Epoch 3/10
2460/2460 [==============================] - 19s 8ms/step - loss: 1.2043 - accuracy: 0.4901 - val_loss: 1.1726 - val_accuracy: 0.5044
Epoch 4/10
2460/2460 [==============================] - 18s 7ms/step - loss: 1.2000 - accuracy: 0.4933 - val_loss: 1.1726 - val_accuracy: 0.5057
Epoch 5/10
2460/2460 [==============================] - 17s 7ms/step - loss: 1.1972 - accuracy: 0.4952 - val_loss: 1.1675 - val_accuracy: 0.5096
Epoch 6/10
2460/2460 [==============================] - 17s 7ms/step - loss: 1.1946 - accuracy: 0.4967 - val_loss: 1.1629 - val_accuracy: 0.5107
Epoch 7/10
2460/2460 [==============================] - 18s 7ms/step - loss: 1.1932 - accuracy: 0.4982 - val_loss: 1.1617 - val_ac

In [52]:
model.evaluate(x_test_1,y_test_1)

2733/2733 [==============================] - 13s 5ms/step - loss: 1.1622 - accuracy: 0.5116


[1.162152647972107, 0.51155024766922]

In [65]:
import sklearn
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score

In [60]:
y_pred_lstm = model.predict(x_test_1)
y_pred_lstm

2733/2733 [==============================] - 8s 3ms/step


array([[0.00523864, 0.05776334, 0.25920367, 0.5552075 , 0.1225869 ],
       [0.01195071, 0.0753161 , 0.23162916, 0.48053995, 0.20056418],
       [0.02424186, 0.12084214, 0.23309943, 0.5005316 , 0.12128499],
       ...,
       [0.25771162, 0.36759347, 0.19373894, 0.15229104, 0.02866495],
       [0.01371712, 0.01922659, 0.04059695, 0.18737102, 0.7390883 ],
       [0.05579996, 0.12489208, 0.17032905, 0.40367025, 0.24530861]],
      dtype=float32)

In [82]:
accuracy_score(np.argmax(y_test_1, axis=1), np.argmax(y_pred_lstm, axis=1))

0.5115502493024107

In [81]:
precision, recall, fscore, support = score(np.argmax(y_test_1, axis=1), np.argmax(y_pred_lstm, axis=1))

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))
print('################################')
print(sklearn.metrics.classification_report(np.argmax(y_test_1, axis=1), np.argmax(y_pred_lstm, axis=1)))

precision: [0.58450894 0.46712907 0.45321512 0.48582945 0.64355444]
recall: [0.57011394 0.5455872  0.23653696 0.62197138 0.57486771]
fscore: [0.5772217  0.50331894 0.31084249 0.54553491 0.60727502]
support: [11410 19874 17938 24805 13417]
################################
              precision    recall  f1-score   support

           0       0.58      0.57      0.58     11410
           1       0.47      0.55      0.50     19874
           2       0.45      0.24      0.31     17938
           3       0.49      0.62      0.55     24805
           4       0.64      0.57      0.61     13417

    accuracy                           0.51     87444
   macro avg       0.53      0.51      0.51     87444
weighted avg       0.51      0.51      0.50     87444



# GRU training

In [56]:
modelGRU = Sequential()
modelGRU.add(GRU(128, return_sequences=True, input_shape=(timesteps, input_dim)))
modelGRU.add(Dropout(0.5))
modelGRU.add(GRU(64))
modelGRU.add(Dropout(0.5))
modelGRU.add(Dense(5, activation='softmax'))

In [57]:
modelGRU.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [58]:
modelGRU.fit(x_train_1, y_train_1, epochs=10, batch_size=256, validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', patience=7, min_delta=0.0001)])

Epoch 1/10
2460/2460 [==============================] - 55s 8ms/step - loss: 1.2525 - accuracy: 0.4605 - val_loss: 1.1878 - val_accuracy: 0.4956
Epoch 2/10
2460/2460 [==============================] - 16s 7ms/step - loss: 1.2123 - accuracy: 0.4855 - val_loss: 1.1746 - val_accuracy: 0.5034
Epoch 3/10
2460/2460 [==============================] - 16s 7ms/step - loss: 1.2034 - accuracy: 0.4907 - val_loss: 1.1707 - val_accuracy: 0.5053
Epoch 4/10
2460/2460 [==============================] - 17s 7ms/step - loss: 1.1980 - accuracy: 0.4947 - val_loss: 1.1641 - val_accuracy: 0.5099
Epoch 5/10
2460/2460 [==============================] - 17s 7ms/step - loss: 1.1945 - accuracy: 0.4964 - val_loss: 1.1644 - val_accuracy: 0.5106
Epoch 6/10
2460/2460 [==============================] - 17s 7ms/step - loss: 1.1920 - accuracy: 0.4990 - val_loss: 1.1609 - val_accuracy: 0.5128
Epoch 7/10
2460/2460 [==============================] - 17s 7ms/step - loss: 1.1905 - accuracy: 0.4998 - val_loss: 1.1613 - val_ac

In [59]:
modelGRU.evaluate(x_test_1,y_test_1)

2733/2733 [==============================] - 14s 5ms/step - loss: 1.1603 - accuracy: 0.5122


[1.1603310108184814, 0.5121563673019409]

In [61]:
y_pred_gru = modelGRU.predict(x_test_1)
y_pred_gru

2733/2733 [==============================] - 8s 3ms/step


array([[0.00734491, 0.07237477, 0.28278   , 0.49905872, 0.13844155],
       [0.01277134, 0.07202061, 0.22959799, 0.451347  , 0.23426305],
       [0.01963235, 0.11098276, 0.22735497, 0.50623524, 0.13579467],
       ...,
       [0.20254816, 0.4044621 , 0.20778607, 0.15882961, 0.02637403],
       [0.01085381, 0.01546   , 0.04705309, 0.18797491, 0.7386582 ],
       [0.0384312 , 0.1166635 , 0.19651674, 0.4289197 , 0.21946889]],
      dtype=float32)

In [83]:
accuracy_score(np.argmax(y_test_1, axis=1), np.argmax(y_pred_gru, axis=1))

0.5121563514935273

In [84]:
precision, recall, fscore, support = score(np.argmax(y_test_1, axis=1), np.argmax(y_pred_gru, axis=1))

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))
print('################################')
print(sklearn.metrics.classification_report(np.argmax(y_test_1, axis=1), np.argmax(y_pred_gru, axis=1)))

precision: [0.5655201  0.4756402  0.44422424 0.49179527 0.63208685]
recall: [0.59798422 0.5177619  0.26240384 0.60894981 0.58582395]
fscore: [0.58129925 0.49580804 0.3299222  0.54413804 0.60807674]
support: [11410 19874 17938 24805 13417]
################################
              precision    recall  f1-score   support

           0       0.57      0.60      0.58     11410
           1       0.48      0.52      0.50     19874
           2       0.44      0.26      0.33     17938
           3       0.49      0.61      0.54     24805
           4       0.63      0.59      0.61     13417

    accuracy                           0.51     87444
   macro avg       0.52      0.51      0.51     87444
weighted avg       0.51      0.51      0.50     87444

